---
  title: "Annotations en Java"
  description: "Définition et traitement simple d'annotations personnalisées en Java."
  categories: 
    - Java
    - I311
    - Lecture
    - Annotations  
---

En java la annotations sont un moyen d'associer des meta-données  aux classes, méthodes, propriétés, paramètres, variables et ... aux annotations. 

Ces méta-données peuvent être utilisés lors d'une traitement du code source, lors de la compilation ou pendant l'exécution. Par exemple pour détecter ou ignorer des erreurs lors de la compilation, générer de la documentation, du code ou des fichiers. 

## Annotations standards

Il existe des annotations standards [Deprecated](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/Deprecated.html), [FunctionalInterface](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/FunctionalInterface.html), [Override](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/Override.html), [SuppressWarings](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/SuppressWarnings.html) et [SafeVarargs](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/SafeVarargs.html).

## Définition d'une annotation

Une annotation se déclare dans un fichier qui a le même nom comme une classe ou une interface.

In [1]:
public @interface MyAnnotation { }

---

Elle peut alors être utilisée partout sur une classe.

In [2]:
@MyAnnotation
public class MyClass {
    @MyAnnotation
    private int x;
    
    @MyAnnotation
    public void m(@MyAnnotation int y) {
        @MyAnnotation
        String m="test";
    }
}

## Paramètres

Si l'on y ajoute des méthodes leurs types de retour (primitifs, référence vers une instance de la class Class, annotation, enumération ou un tableau des précédents) et leurs noms définissent respectivement les paramètres de l'annotation. Il est aussi possible de donner des valeurs par défaut.  

In [3]:
public @interface MyAnnotation {
    public enum Digit {UN, DEUX};
    
    String message();
    int size() default 10;
    Digit digit();
    Class[] classes();
}

---

In [4]:
@MyAnnotation(message="hello", digit=MyAnnotation.Digit.UN, classes={String.class, Arrays.class})
public class MyClass {
}

## Value

Si leur seul attribut obligatoire s'appelle `value`, il est possible d'omettre son nom lors de l'annotation.

In [5]:
public @interface MyAnnotation {
    String value();
    int size() default 10;
}

In [6]:
@MyAnnotation("hello")
public class MyClass { }

## Portée

Une déclaration d'annotation peut être annotée avec [Target](https://docs.oracle.com/en/java/javase/17/docs/api/java.base/java/lang/annotation/Target.html) pour contrôler ce qui peut être annoté :  ANNOTATION_TYPE, CONSTRUCTOR, FIELD, LOCAL_VARIABLE, METHOD, PACKAGE, MODULE, PARAMETER, TYPE, et TYPE_PARAMETER

In [7]:
//| echo: false

In [8]:
import java.lang.annotation.Target;
import java.lang.annotation.ElementType;

In [9]:
@Target({ElementType.LOCAL_VARIABLE, ElementType.FIELD})
public @interface MyAnnotation {
    int value() default 0;
}

---

In [10]:
public class MyClass {    
    @MyAnnotation
    int x;
    public void m() {@MyAnnotation int x;}
}

In [11]:
@MyAnnotation
public class MyClass {    
    int x;
}

CompilationException: 

## Annotations répétables 1/2

Par défaut, une annotation ne peut pas être utilisée plusieurs fois sur un même élément.
Il faut alors soit définir un wrapper (une annotations qui peut les contenir) :

In [12]:
@Target({ElementType.LOCAL_VARIABLE, ElementType.FIELD})
public @interface MyAnnotations {
    MyAnnotation[] value();
}

In [13]:
public class MyClass {    
    @MyAnnotations({@MyAnnotation(1),@MyAnnotation(2)})
    int x;
}

## Annotations répétables 2/2

ou les autoriser à être répétable avec `@Repeatable` pour éviter la déclaration explicite du tableau d'annotations.

In [14]:
//| echo: false
import java.lang.annotation.Repeatable;

In [15]:
@Repeatable(MyAnnotations.class)
@Target({ElementType.LOCAL_VARIABLE, ElementType.FIELD})
public @interface MyAnnotation {
    int value() default 0;
}

In [16]:
public class MyClass {    
    @MyAnnotation(1)
    @MyAnnotation(2)
    int x;
}

## Package

Il est aussi possible d'annoter un package dans le fichier `package-info.java`.

## Durée de vie (Retention)

L'annotation `@Retention` permet de préciser si l'annotation n'est accessible que pendant la compilation (SOURCE), si en plus elle est ajoutée au fichier .class (CLASS) ou si elle est accessible durant l'exécution (RUNTIME).

L'annotation `@Inherited` permet d'indiquer que l'annotation d'un type est associée aussi à ses descendants et l'annotation `@Documented` qu'elle doit apparaitre dans la javadoc.

In [17]:
//| echo: false
import java.lang.annotation.Documented;
import java.lang.annotation.ElementType;
import java.lang.annotation.Inherited;
import java.lang.annotation.Retention;
import java.lang.annotation.RetentionPolicy;
import java.lang.annotation.Target;

In [18]:
@Documented
@Target(ElementType.METHOD)
@Inherited
@Retention(RetentionPolicy.RUNTIME)
public @interface MyAnnotation {
  public enum Status {IN_PROGRESS, DONE} 
  Status status() default MyAnnotation.Status.DONE;
}

---

In [19]:
public class MyClass {    
    @MyAnnotation
    public void m1() {System.out.println("Hello1");}

    @MyAnnotation(status=MyAnnotation.Status.IN_PROGRESS)
    public void m2() {System.out.println("Hello2");}
    
    @MyAnnotation(status=MyAnnotation.Status.DONE)
    public void m3() {System.out.println("Hello3");}

    public void m4() {System.out.println("Hello4");}
}

## Utilisation par instrospection (1/2)

Les annotations peuvent être utilisées lors de l'exécution par introspection à condition d'avoir la rétention adaptée. 

Pour rappel, la class Class permet de représenter une classe et d'accéder à la représentation de ses constituants représentés chacun par une classe spécifique (Method, Field, Constructor, ...). Ensuite des actions peuvent être déclenchées de façon programmatique par exemple invoquer une méthode sur une instance, invoquer un constructeur, ...

L'exemple ci-dessus affiche les méthodes de la classe précédente annotées avec `@MyAnnotation`.

---

In [20]:
//| echo: false
import java.lang.reflect.Method;

In [21]:
MyClass myObject = new MyClass();
Arrays.stream(myObject.getClass().getDeclaredMethods())
    .filter(m->m.getAnnotation(MyAnnotation.class)!=null)
    .map(m->"La méthode %s a comme statut %s.".formatted(m.getName(),m.getAnnotation(MyAnnotation.class).status()))
    .forEach(System.out::println);

La méthode m2 a comme statut IN_PROGRESS.


La méthode m1 a comme statut DONE.


La méthode m3 a comme statut DONE.


## Utilisation par instrospection (2/2)

Il est aussi possible d'invoquer dynamiquement ces méthodes.

In [22]:
//| echo: false
import java.lang.reflect.Method;

---

In [23]:
MyClass myObject = new MyClass();
Arrays.stream(myObject.getClass().getDeclaredMethods())
    .filter(m->m.getAnnotation(MyAnnotation.class)!=null &&
               m.getAnnotation(MyAnnotation.class).status().equals(MyAnnotation.Status.DONE))
    .forEach(m->{
        try {m.invoke(myObject);} 
        catch (Exception e) {}
        });

Hello1


Hello3
